In [1]:
%config InlineBackend.figure_format = 'retina'

import pandas
from IPython.display import display, HTML

display(HTML("<style>.container{ width:100% !important; }</style>"))

df = src_df = pandas.read_csv('libavif.csv')
df['size_kb'] = df['size'] / 1000
df['data'] = df.agg('<tt>{0[size_kb]:5.2f}&nbsp;<br>{0[perf]:5.2f}&nbsp;<br>{0[dssim]:7.3f}</tt>'.format, axis=1)

df

,fname,codec,effort,quality,outname,size,enc_time,perf,dssim,size_kb,data
0,munhell,libavif,5,10,munhell.s5.q10,20427,0.372140,0.422602,0.002127,20.427,<tt>20.43&nbsp;<br> 0.42&nbsp;<br> 0.002</tt>
1,munhell,webp,5,10,munhell.q97.5,17664,0.032048,4.907308,0.002141,17.664,<tt>17.66&nbsp;<br> 4.91&nbsp;<br> 0.002</tt>
2,munhell,libavif,7,10,munhell.s7.q10,24763,0.159725,0.984613,0.002286,24.763,<tt>24.76&nbsp;<br> 0.98&nbsp;<br> 0.002</tt>
3,munhell,webp,7,10,munhell.q97,17404,0.032376,4.857474,0.002341,17.404,<tt>17.40&nbsp;<br> 4.86&nbsp;<br> 0.002</tt>
4,munhell,libavif,5,12,munhell.s5.q12,19683,0.406752,0.386641,0.002624,19.683,<tt>19.68&nbsp;<br> 0.39&nbsp;<br> 0.003</tt>
...,...,...,...,...,...,...,...,...,...,...,...
427,parrots,webp,7,42,parrots.q29.5,11978,0.040383,9.737257,0.683859,11.978,<tt>11.98&nbsp;<br> 9.74&nbsp;<br> 0.684</tt>
428,parrots,libavif,5,44,parrots.s5.q44,7077,0.475950,0.826172,0.751434,7.077,<tt> 7.08&nbsp;<br> 0.83&nbsp;<br> 0.751</tt>
429,parrots,webp,5,44,parrots.q25,10950,0.041057,9.577294,0.756441,10.950,<tt>10.95&nbsp;<br> 9.58&nbsp;<br> 0.756</tt>
430,parrots,libavif,7,44,parrots.s7.q44,8065,0.083293,4.720866,0.803646,8.065,<tt> 8.06&nbsp;<br> 4.72&nbsp;<br> 0.804</tt>


In [2]:
display(HTML("""<table>
<tr><th>Each cell:
<tr><td>
Size in kb<br>
Megapixels/s<br>
DSSIM mectric
</table>
"""))

tables = []

for fname in src_df.fname.unique():
    df = src_df[src_df.fname == fname]
    df = df.pivot(index='quality', columns=['codec', 'effort'], values='data')
    tables.append(f"<h1>{fname}</h1>" + df.to_html(escape=False))

display(HTML("".join(
    f'<div style="float:left">{table}</div>'
    for table in tables
)))

codec,libavif,webp,libavif,webp
effort,5,5,7,7
quality,,,,
10,20.43 0.42 0.002,17.66 4.91 0.002,24.76 0.98 0.002,17.40 4.86 0.002
12,19.68 0.39 0.003,16.94 5.32 0.003,23.71 1.01 0.003,16.94 5.32 0.003
14,19.08 0.38 0.003,16.44 5.01 0.003,22.96 1.10 0.004,16.01 4.86 0.004
16,18.52 0.42 0.004,16.01 4.86 0.004,22.00 1.09 0.004,15.54 5.37 0.004
18,18.09 0.43 0.004,15.54 5.37 0.004,21.26 1.09 0.006,14.76 5.53 0.005
20,17.71 0.40 0.005,15.17 4.74 0.005,20.65 0.94 0.007,14.26 4.76 0.007
22,17.39 0.37 0.006,14.76 5.53 0.005,20.00 0.96 0.008,13.80 4.54 0.008
24,17.02 0.41 0.007,14.26 4.76 0.007,19.64 1.09 0.010,13.42 5.20 0.010
